# Automating data import with python Pandas, loading into a postgres database, and visualizing with Power BI.

In [8]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

In [30]:
import os
print(os.getcwd())

D:\automating-data-imports-with-python


In [41]:
# Load .env file — force override in case it was cached before
load_dotenv(dotenv_path="D:/automating-data-imports-with-python/.env.txt", override=True)

# Fetch environment variables
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB')  # renamed for clarity

# Print them to verify (optional; remove sensitive ones in real usage)
print("DB_USER:", DB_USER)
print("DB_PASSWORD:", DB_PASSWORD)  # be careful with this in real apps
print("DB_HOST:", DB_HOST)
print("DB_PORT:", DB_PORT)
print("DB_NAME:", DB_NAME)


DB_USER: postgres
DB_PASSWORD: 1234
DB_HOST: localhost
DB_PORT: 5432
DB_NAME: breast_cancer


* Import the CSV file and load it into a pandas dataframe.

In [52]:
def create_dataframe(path):
    df = pd.read_csv(path)
    return df
    

* HIghlight the structure and nature of the dataset.

In [10]:
def check_dataset(dataframe,head=5):
    print(20*"#", "HEAD", 20*"#")
    print(dataframe.head(head))
    print(20*"#", "TAIL", 20*"#")
    print(dataframe.tail(head))
    print(20*"#", "SHAPE", 20*"#")
    print(dataframe.shape)
    print(20*"#", "TYPES", 20*"#")
    print(dataframe.dtypes)
    print(20*"#", "NA", 20*"#")
    print(dataframe.isnull().sum())
    print(20*"#", "QUARTILES", 20*"#")
    print(dataframe.describe([0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.99]).T)

In [44]:
def clean_dataset(dataframe):
    clean_data = dataframe
    return clean_data

In [50]:
def load_dataset(dataframe):
    try:
        engine = create_engine(
            f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
        )
        
        # Optional: check connection
        with engine.connect() as conn:
            print("✅ Database connection successful.")

        # Call your cleaning function
        clean_data = clean_dataset(dataframe)

        # Load to PostgreSQL
        clean_data.to_sql('patients', engine, if_exists='replace', schema='patients')

        print("✅ Dataset successfully loaded.")
    
    except Exception as e:
        print(f"❌ Error loading dataset: {e}")

In [53]:
breast_cancer = "Breast_Cancer.csv"
df = create_dataframe(breast_cancer)

In [55]:
check_dataset(df)

#################### HEAD ####################
   Age   Race Marital Status T Stage  N Stage 6th Stage  \
0   68  White        Married       T1      N1       IIA   
1   50  White        Married       T2      N2      IIIA   
2   58  White       Divorced       T3      N3      IIIC   
3   58  White        Married       T1      N1       IIA   
4   47  White        Married       T2      N1       IIB   

               differentiate Grade   A Stage  Tumor Size Estrogen Status  \
0      Poorly differentiated     3  Regional           4        Positive   
1  Moderately differentiated     2  Regional          35        Positive   
2  Moderately differentiated     2  Regional          63        Positive   
3      Poorly differentiated     3  Regional          18        Positive   
4      Poorly differentiated     3  Regional          41        Positive   

  Progesterone Status  Regional Node Examined  Reginol Node Positive  \
0            Positive                      24                      1 

In [54]:
load_dataset(df)

✅ Database connection successful.
✅ Dataset successfully loaded.
